In [1]:
pip install torch numpy



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - 256, (64,))
    x = torch.stack([data[i:i+256] for i in ix])
    y = torch.stack([data[i+1:i+257] for i in ix])
    return x.to(device), y.to(device)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(200)
        for k in range(200):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(384, head_size, bias=False)
        self.query = nn.Linear(384, head_size, bias=False)
        self.value = nn.Linear(384, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(256, 256)))
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        return wei @ v

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, 384)
        self.dropout = nn.Dropout(0.2)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(out))

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(0.2),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        return x + self.ffwd(self.ln2(x))

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, 384)
        self.position_embedding_table = nn.Embedding(256, 384)
        self.blocks = nn.Sequential(*[Block(384, n_head=6) for _ in range(6)])
        self.ln_f = nn.LayerNorm(384)
        self.lm_head = nn.Linear(384, vocab_size)
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            logits = logits.view(B*T, logits.shape[-1])
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):``
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -256:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPTLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
for iter in range(5000):
    if iter % 500 == 0 or iter == 4999:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


10.788929 M parameters
step 0: train loss 4.2227, val loss 4.2305
step 500: train loss 1.7550, val loss 1.9113
step 1000: train loss 1.3944, val loss 1.6093
step 1500: train loss 1.2682, val loss 1.5332
step 2000: train loss 1.1863, val loss 1.5096
step 2500: train loss 1.1284, val loss 1.4972
step 3000: train loss 1.0708, val loss 1.5016
step 3500: train loss 1.0163, val loss 1.5174
step 4000: train loss 0.9638, val loss 1.5236
step 4500: train loss 0.9131, val loss 1.5555
step 4999: train loss 0.8551, val loss 1.5667

Her gyouthful cheeks a little flower,
For thy law agozing promises: shall I'll tutor
The ohergeous pluck of on the fust, I move
A man that have heard him, and as he dies,
As we are hungry as the people's vice
As instrumes currently.
 Canstillow!

GLOUCESTER:
Belike not me; at what I should be?

LADY ANNE:
None, my husband, let me lose do thee.

GLOUCESTER:
By my much open in twis, but in my secons.
First Kathard: I will be satisfied,
Who, God dead, fortunes, I love myse

In [2]:
import torch
import pickle

# Save the model
torch.save(model.state_dict(), 'gpt_model.pth')

# Save tokenizer mappings
tokenizer = {'stoi': stoi, 'itos': itos}
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

print("Model and tokenizer saved successfully!")


Model and tokenizer saved successfully!


In [3]:
import torch
import pickle

# Load the tokenizer
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
    stoi = tokenizer['stoi']
    itos = tokenizer['itos']

# Reinitialize the model
model = GPTLanguageModel()
model.load_state_dict(torch.load('gpt_model.pth'))
model.to(device)
model.eval()  # Set to evaluation mode

# Tokenization functions
encode = lambda s: [stoi[c] for c in s]  
decode = lambda l: ''.join([itos[i] for i in l])  

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


In [14]:
def generate_text(prompt, max_tokens=500):
    model.eval() 
    context = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
    generated_idx = model.generate(context, max_new_tokens=max_tokens)[0].tolist()
    return decode(generated_idx)

prompt = "His"
output = generate_text(prompt, max_tokens=200)
print("Generated Text:\n", output)


Generated Text:
 His prayers he hath desert you here all forget
Myself to your highness: consider so he
Was not more shamed to him; therefore, with his sudden,
And purpose himself him that trual should be extracted
Of i


In [12]:
pip install huggingface



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
